<a href="https://colab.research.google.com/github/UinspireHuaxia/BandInMuseScore/blob/master/Introduction_to_catsim_by_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example usages

![](https://douglasrizzo.com.br/catsim/_static/logo_text.svg)

This is a tutorial for [catsim](https://github.com/douglasrizzo/catsim/), a Python package which allows users to simulate computerized adaptive tests or to use in their own applications with the purpose of automating adaptive tests.

This tutorial was originally developed as a notebook on Google Colab, so, if for any reason you are seeing this outside of Colab, you can go back to Colab, copy it and test it yourself using [this link](https://colab.research.google.com/drive/14zEWoDudBCXF0NO-qgzoQpWUGBcJ2lPH?usp=sharing).

In this notebook, I'll exemplify both approaches. The documentation of all modules and functions used here are available in the [catsim website](https://douglasrizzo.com.br/catsim/index.html).

First, let's install catsim and import the relevant modules:

In [ ]:
!pip install -U catsim

In [ ]:
# this function generates an item bank, in case the user cannot provide one
from catsim.cat import generate_item_bank
# simulation package contains the Simulator and all abstract classes
from catsim.simulation import *
# initialization package contains different initial proficiency estimation strategies
from catsim.initialization import *
# selection package contains different item selection strategies
from catsim.selection import *
# estimation package contains different proficiency estimation methods
from catsim.estimation import *
# stopping package contains different stopping criteria for the CAT
from catsim.stopping import *
import catsim.plot as catplot
from catsim.irt import icc

import random

import matplotlib.pyplot as plt

## Generating an item bank

The `generate_item_bank()` function provides a convenient way to generate realistic item parameter matrices from probability distributions.

In [ ]:
bank_size = 5000
items = generate_item_bank(bank_size,itemtype='3PL')
catplot.gen3d_dataset_scatter(items)

### Visualizing parameter distribution

`generate_item_bank()` returns a numpy.ndarray with 4 columns, corresponding to the discrimination, difficulty, guessing and upper-asymptote parameters of the 4-parameter logistic model of Item Response Theory.

We can plot and visualize their distributions like so:

In [ ]:
catplot.param_dist(items, figsize=(9,7))

### Visualizing individual items

catsim also provides a function to plot the characteristic curve of an item. Notice how this item has been generated according to the 4-parameter logistic model of the Item Response Theory. Item banks under different logistic models can be generated by changing the `itemtype` parameter of `generate_item_bank()`.

In [ ]:
a, b, c, d = items[0]
catplot.item_curve(a,b,c,d)

## Running simulations

A simulation requires the following objects:

- an item parameter matrix
- a proficiency initializer, which sets the initial $\theta$ values for examinees
- an item selector, which selects items to be applied to examinees according to some rule
- a proficiency estimator, which estimates the new $\theta$ values for examinees after an item is answered
- a "stopper", an object which checks if the test must be stopped according to some rule

We have already created an item parameter matrix, so let's go ahead and create the other objects...

In [ ]:
initializer = RandomInitializer()
selector = MaxInfoSelector()
estimator = NumericalSearchEstimator()
stopper = MaxItemStopper(20)

catsim provides different options for each of the aforementioned types of objects in the following modules

- `catsim.simulation`
- `catsim.initialization`
- `catsim.selection`
- `catsim.estimation`

Each module also provides an abstract base class which can be inherited in order to create new methods that can be used in the simulation process.

### Creating a simulator

The Simulator is the object that takes all of the objects created previously and executes a CAT simulation. To represent the examinees, The Simulator can receive either an integer, which will be converted to a normal distribution, or a 1D `numpy.ndarray`, whose values will be used as the proficiencies of the examinees.

Here we will use an integer.

In [ ]:
s = Simulator(items, 10, initializer, selector, estimator, stopper)

### Starting the simulation

To execute the simulations, call the simulate() method of the Simulator object.

In [ ]:
s.simulate(verbose=True)

### Acessing simulation results

After the simulation if over, information is provided through the attributes of the Simulator:

In [ ]:
print('Bias:', s.bias)
print('Mean squared error:', s.mse)
print('Root mean squared error:', s.rmse)

Information for individual examinees can also be accessed through the attributes of the Simulator.

In [ ]:
examinee_index = 0
print('Accessing examinee', examinee_index, 'results...')
print('    True proficiency:', s.examinees[examinee_index])
print('    Items administered:', s.administered_items[examinee_index])
print('    Responses:', s.response_vectors[examinee_index])
print('    Proficiency estimation during each step of the test:', s.estimations[examinee_index])

The test progress for an individual examinee can also be plotted using the `catsim.plot.test_progress(function)`. The amount of information in the chart can be tuned using different arguments for the function.

In [ ]:
catplot.test_progress(simulator=s,index=0)

### Simulation example 2

This example uses a `numpy.ndarray` to represent examinees. We will also plot more information than before in our test progress plot, adding test information to it.

In [ ]:
examinees = numpy.random.normal(size=10)
s = Simulator(items, examinees, RandomInitializer(), MaxInfoSelector(), NumericalSearchEstimator(), MinErrorStopper(.3))
s.simulate(verbose=True)
catplot.test_progress(simulator=s,index=0, info=True)

### Simulating non-adaptive tests

catsim can also simulate linear (non-adaptive) tests by using a linear item selector. The linear selector receives the item indices as arguments, retrieves them from the item parameter matrix and applies them in order to all examinees.

In [ ]:
indexes = numpy.random.choice(items.shape[0], 50, replace=False)
print('The following items will be applied to the examinees in this order:', indexes)
s = Simulator(items, 10, RandomInitializer(), LinearSelector(indexes), NumericalSearchEstimator(), MaxItemStopper(50))
s.simulate(verbose=True)

Here, we will also plot the estimation error for an examinee's $\hat\theta$ value during the progress of the test.

In [ ]:
catplot.test_progress(simulator=s,index=0, info=True, see=True)

## Using catsim objects outside of a Simulator

The objects provided by catsim can also be used directly, outside of a simulation. This allows users to use these objects in their own software, to power their own CAT applications.


Let's pretend we are in the middle of a test application and create some dummy data for an examinee, as well as some objects we will use to select the next item for this examinee, re-estimate their proficiency and check if the test should be stopped or if a new item should be applied to the examinee.

In [ ]:
responses = [True, True, False, False]
administered_items = [1435, 3221, 17, 881]

initializer = FixedPointInitializer(0)
selector = MaxInfoSelector()
estimator = NumericalSearchEstimator()
stopper = MinErrorStopper(.2)

This dummy data means that the examinee has answered items 1435, 3221, 17 and 881 from our item bank (generated at the start of this notebook). They have answered the first two items correctly (represented by the `True` values in the `responses` list) and two items incorrectly (the last values in the list).

### Initializing $\hat\theta$

Even though this information is already enough to estimate the current proficiency of the examinee, we'll go ahead and use our initializer to estimate a dummy initial proficiency anyway.

In [ ]:
est_theta = initializer.initialize()
print('Examinee initial proficiency:', est_theta)

### Estimating a new $\hat\theta$

Now, we will use the answers the examinee has given so far (remember, we're pretending they have already answered a few items) to estimate a more precise $\hat\theta$ proficiency for them.

Internally, the estimator uses the item bank and the indices of the administered items to get the relevant item parameters, then uses the response vector to know which items the examinee has answered correctly and incorrectly to generate the new value for $\hat\theta$.

Some estimators may or may not use the current value of $\hat\theta$ to speed up estimation.

**After getting to the end of the notebook, come back to this cell to simulate a new item being applied to this examinee**.

In [ ]:
new_theta = estimator.estimate(items=items, administered_items=administered_items, response_vector=responses, est_theta=est_theta)
print('Estimated proficiency, given answered items:', new_theta)

### Checking whether the test should end

We do not know whether the CAT should select another item to the examinee or if the test should end. The stoper will give us this answer through the `stop()` method.

In [ ]:
_stop = stopper.stop(administered_items=items[administered_items], theta=est_theta)
print('Should the test be stopped:', _stop)

### Selecting a new item

The selector takes the item parameter matrix and the current $\hat\theta$ value to select the new item the examinee will answer. It uses the indices of administered items to ignore them.

In [ ]:
item_index = selector.select(items=items, administered_items=administered_items, est_theta=est_theta)
print('Next item to be administered:', item_index)

### Simulating a response

In order to apply the next item, we need to pretend here that the examinee has answered an item. In the real world, this information could be fetched by an external application, but here we will use IRT to simulate the answer probabilistically.

For that, we will use a hypothetical true proficiency value of $\theta$. It is important that this value is fixed, as it represents the abilities of a person when taking the test.

(By the way, this is exactly what the Simulator does during simulations.)

In [ ]:
true_theta = 0.8
a, b, c, d = items[item_index]
prob = icc(true_theta, a, b, c, d)
correct = prob > random.uniform(0, 1)

print('Probability to correctly answer item:', prob)
print('Did the user answer the selected item correctly?', correct)

Finally, we add the index of the administered item to the examinee and their answer to the item to our lists and we are ready for the next step of the adaptive test.

Go back to the **"Estimating a new $\hat\theta$"** step above to simulate another step of the CAT.

In [ ]:
administered_items.append(item_index)
responses.append(correct)